## Бонусное задание: word2vec на PyTorch

Как вы уже могли заметить, идея, лежащая в основе [word2vec](https://arxiv.org/pdf/1310.4546), достаточно общая. В данном задании вы реализуете его самостоятельно.

Дисклеймер: не стоит удивляться тому, что реализация от `gensim` (или аналоги) обучается быстрее и работает точнее. Она использует множество доработок и ускорений, а также достаточно эффективный код. Ваша задача добиться промежуточных результатов за разумное время.

P.s. Как ни странно, GPU в этом задании нам не потребуется.

__Requirements:__ if you're running locally, in the selected environment run the following command:

```pip install --upgrade nltk bokeh umap-learn```


In [1]:
!pip install --upgrade nltk bokeh umap-learn

   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.9 MB 1.4 MB/s eta 0:00:06
   ---------------------------------------- 0.1/6.9 MB 825.8 kB/s eta 0:00:09
    --------------------------------------- 0.1/6.9 MB 1.3 MB/s eta 0:00:06
   - -------------------------------------- 0.2/6.9 MB 1.6 MB/s eta 0:00:05
   -- ------------------------------------- 0.4/6.9 MB 2.4 MB/s eta 0:00:03
   ----- ---------------------------------- 0.9/6.9 MB 3.8 MB/s eta 0:00:02
   ------- -------------------------------- 1.3/6.9 MB 4.7 MB/s eta 0:00:02
   ------- -------------------------------- 1.3/6.9 MB 4.7 MB/s eta 0:00:02
   --------- ------------------------------ 1.7/6.9 MB 4.5 MB/s eta 0:00:02
   ---------------- ----------------------- 2.9/6.9 MB 7.1 MB/s eta 0:00:01
   -------------------- ------------------- 3.4/6.9 MB 7.6 MB/s eta 0:00:01
   ----------------------- ---------------- 4.0/6.9 MB 8.0 MB/s eta 0:00:01
   --------------

In [2]:
import itertools
import random
import string
from collections import Counter
from itertools import chain

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import umap
from IPython.display import clear_output
from matplotlib import pyplot as plt
from nltk.tokenize import WordPunctTokenizer
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from tqdm.auto import tqdm as tqdma

In [3]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
data = list(open("./quora.txt", encoding="utf-8"))
data[0]

"Can I get back with my ex even though she is pregnant with another guy's baby?\n"

Токенизация – первый шаг.
Тексты, с которыми мы работаем, включают в себя пунктуацию, смайлики и прочие нестандартные токены, так что простой `str.split` не подойдет.

Обратимся к `nltk` - библиотеку, нашла широкое применеие в области NLP.

In [8]:
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[0]))

['Can', 'I', 'get', 'back', 'with', 'my', 'ex', 'even', 'though', 'she', 'is', 'pregnant', 'with', 'another', 'guy', "'", 's', 'baby', '?']


In [12]:
data_tok = [
    tokenizer.tokenize(
        line.translate(str.maketrans("", "", string.punctuation)).lower()
    )
    for line in data
]
data_tok = [x for x in data_tok if len(x) >= 3]
data_tok

[['can',
  'i',
  'get',
  'back',
  'with',
  'my',
  'ex',
  'even',
  'though',
  'she',
  'is',
  'pregnant',
  'with',
  'another',
  'guys',
  'baby'],
 ['what',
  'are',
  'some',
  'ways',
  'to',
  'overcome',
  'a',
  'fast',
  'food',
  'addiction'],
 ['who',
  'were',
  'the',
  'great',
  'chinese',
  'soldiers',
  'and',
  'leaders',
  'who',
  'fought',
  'in',
  'ww2'],
 ['what', 'are', 'zip', 'codes', 'in', 'the', 'bay', 'area'],
 ['why',
  'was',
  'george',
  'rr',
  'martin',
  'critical',
  'of',
  'jk',
  'rowling',
  'after',
  'losing',
  'the',
  'hugo',
  'award'],
 ['what', 'can', 'i', 'do', 'to', 'improve', 'my', 'immune', 'system'],
 ['how', 'is', 'your', 'relationship', 'with', 'your', 'mother', 'in', 'law'],
 ['how', 'does', 'one', 'get', 'free', 'psn', 'codesvita', 'codes'],
 ['what', 'is', 'your', 'review', 'of', 'osquery'],
 ['how', 'can', 'i', 'look', 'smart', 'and', 'act', 'smart'],
 ['which',
  'brand',
  'should',
  'go',
  'with',
  'the',
  'gtx'

Несколько проверок:

In [13]:
assert all(
    isinstance(row, (list, tuple)) for row in data_tok
), "please convert each line into a list of tokens (strings)"
assert all(
    all(isinstance(tok, str) for tok in row) for row in data_tok
), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all("a" <= x.lower() <= "z" for x in tok)
assert all(
    map(lambda l: not is_latin(l) or l.islower(), map(" ".join, data_tok))
), "please make sure to lowercase the data"

Ниже заданы константы ширины окна контекста и проведена предобработка для построения skip-gram модели.

In [14]:
min_count = 5
window_radius = 5

In [18]:
vocabulary_with_counter = Counter(chain.from_iterable(data_tok))

word_count_dict = dict()
for word, counter in vocabulary_with_counter.items():
    if counter >= min_count:
        word_count_dict[word] = counter

vocabulary = set(word_count_dict.keys())
print(vocabulary_with_counter)
del vocabulary_with_counter

Counter({'the': 251943, 'what': 206625, 'is': 185166, 'a': 154689, 'i': 142777, 'to': 141494, 'in': 139185, 'how': 135306, 'of': 111927, 'do': 104139, 'are': 97955, 'and': 88917, 'for': 74743, 'can': 70901, 'you': 58842, 'why': 56107, 'it': 50937, 'my': 45150, 'best': 40904, 'does': 39578, 'on': 37827, 'or': 32751, 'if': 30384, 'have': 30215, 'be': 29573, 'with': 29117, 'which': 28979, 'that': 28224, 'an': 27979, 'some': 27375, 'should': 26453, 'get': 25801, 'from': 25247, 'your': 20570, 'like': 18924, 'when': 18888, 'at': 18218, 'india': 17838, 'good': 17605, 'who': 17207, 'there': 16980, 'will': 16799, 'as': 16785, 'would': 16343, 'people': 16150, 'not': 15258, 'about': 14408, 'between': 13952, 'one': 13333, 'any': 12615, 'me': 12451, 'we': 12440, 'most': 12369, 'where': 12343, 'did': 12113, 'was': 11712, 'by': 11705, 'make': 11399, 'so': 11300, 'this': 11050, 'they': 10672, 'am': 10544, 'after': 10242, 'way': 10224, 'has': 9518, 'use': 9454, 'much': 9388, 'difference': 9268, 'time':

In [19]:
word_count_dict

{'can': 70901,
 'i': 142777,
 'get': 25801,
 'back': 3075,
 'with': 29117,
 'my': 45150,
 'ex': 828,
 'even': 2572,
 'though': 701,
 'she': 3329,
 'is': 185166,
 'pregnant': 788,
 'another': 1743,
 'guys': 1117,
 'baby': 618,
 'what': 206625,
 'are': 97955,
 'some': 27375,
 'ways': 3476,
 'to': 141494,
 'overcome': 810,
 'a': 154689,
 'fast': 998,
 'food': 2182,
 'addiction': 241,
 'who': 17207,
 'were': 4884,
 'the': 251943,
 'great': 1798,
 'chinese': 1652,
 'soldiers': 211,
 'and': 88917,
 'leaders': 199,
 'fought': 118,
 'in': 139185,
 'ww2': 200,
 'zip': 72,
 'codes': 189,
 'bay': 325,
 'area': 1056,
 'why': 56107,
 'was': 11712,
 'george': 275,
 'rr': 12,
 'martin': 125,
 'critical': 151,
 'of': 111927,
 'jk': 57,
 'rowling': 39,
 'after': 10242,
 'losing': 417,
 'hugo': 9,
 'award': 103,
 'do': 104139,
 'improve': 2133,
 'immune': 67,
 'system': 2965,
 'how': 135306,
 'your': 20570,
 'relationship': 1965,
 'mother': 651,
 'law': 1527,
 'does': 39578,
 'one': 13333,
 'free': 3727

In [16]:
word_to_index = {word: index for index, word in enumerate(vocabulary)}
index_to_word = {index: word for word, index in word_to_index.items()}

Пары `(слово, контекст)` на основе доступного датасета сгенерированы ниже.

In [20]:
context_pairs = []

for text in data_tok:
    for i, central_word in enumerate(text):
        context_indices = range(
            max(0, i - window_radius), min(i + window_radius, len(text))
        )
        for j in context_indices:
            if j == i:
                continue
            context_word = text[j]
            if central_word in vocabulary and context_word in vocabulary:
                context_pairs.append(
                    (word_to_index[central_word], word_to_index[context_word])
                )

print(f"Generated {len(context_pairs)} pairs of target and context words.")

Generated 40220313 pairs of target and context words.


#### Подзадача №1: subsampling
Для того, чтобы сгладить разницу в частоте встречаемсости слов, необходимо реализовать механизм subsampling'а.
Для этого вам необходимо реализовать функцию ниже.

Вероятность **исключить** слово из обучения (на фиксированном шаге) вычисляется как
$$
P_\text{drop}(w_i)=1 - \sqrt{\frac{t}{f(w_i)}},
$$
где $f(w_i)$ – нормированная частота встречаемости слова, а $t$ – заданный порог (threshold).

In [85]:
def subsample_frequent_words(word_count_dict, threshold=1e-5):
    """
    Calculates the subsampling probabilities for words based on their frequencies.

    This function is used to determine the probability of keeping a word in the dataset
    when subsampling frequent words. The method used is inspired by the subsampling approach
    in Word2Vec, where each word's frequency affects its probability of being kept.

    Parameters:
    - word_count_dict (dict): A dictionary where keys are words and values are the counts of those words.
    - threshold (float, optional): A threshold parameter used to adjust the frequency of word subsampling.
                                   Defaults to 1e-5.

    Returns:
    - dict: A dictionary where keys are words and values are the probabilities of keeping each word.

    Example:
    >>> word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
    >>> subsample_frequent_words(word_counts)
    {'the': 0.028, 'is': 0.223, 'apple': 1.0}
    """
    s=0
    for i in word_count_dict.values():
        s+=i
    for i in word_count_dict.keys():
        f=word_count_dict[i]/s
        a = 1 - np.sqrt(threshold/f)
        
    ### YOUR CODE HERE
    keep_prob_dict = {
        word: np.sqrt(threshold/(word_count_dict[word]/s)) for word in word_count_dict.keys()
    }  # THIS IS A PLACEHOLDER!
    return keep_prob_dict

In [89]:
keep_prob_dict = subsample_frequent_words(word_count_dict)
keep_prob_dict

{'can': 0.02911670312402145,
 'i': 0.020518199702878066,
 'get': 0.048266978094480766,
 'back': 0.13981244114594021,
 'with': 0.04543547424212828,
 'my': 0.03648711297249418,
 'ex': 0.2694345035269634,
 'even': 0.15287374210909657,
 'though': 0.2928258210670549,
 'she': 0.13437283533713018,
 'is': 0.018017218948841916,
 'pregnant': 0.2761882955870675,
 'another': 0.18570332727278635,
 'guys': 0.23197532514724706,
 'baby': 0.3118703883964877,
 'what': 0.017055990497688175,
 'are': 0.02477165792788909,
 'some': 0.046858816414833686,
 'ways': 0.1315008302423615,
 'to': 0.020611014394515556,
 'overcome': 0.2724117707433109,
 'a': 0.01971236185129397,
 'fast': 0.24541613263254033,
 'food': 0.16597436110020564,
 'addiction': 0.49941301644615815,
 'who': 0.05910385377849528,
 'were': 0.11093805592425104,
 'the': 0.015446042678242946,
 'great': 0.18284097772878674,
 'chinese': 0.1907494775775963,
 'soldiers': 0.5337368043364796,
 'and': 0.026000157449729635,
 'leaders': 0.5495938198653355,
 'f

#### Подзадача №2: negative sampling
Для более эффективного обучения необходимо не только предсказывать высокие вероятности для слов из контекста, но и предсказывать низкие для слов, не встреченных в контексте. Для этого вам необходимо вычислить вероятност использовать слово в качестве negative sample, реализовав функцию ниже.

В оригинальной статье предлагается оценивать вероятность слов выступать в качестве negative sample согласно распределению $P_n(w)$
$$
P_n(w) = \frac{U(w)^{3/4}}{Z},
$$

где $U(w)$ распределение слов по частоте (или, как его еще называют, по униграммам), а $Z$ – нормировочная константа, чтобы общая мера была равна $1$.

In [57]:
def get_negative_sampling_prob(word_count_dict):
    """
    Calculates the negative sampling probabilities for words based on their frequencies.

    This function adjusts the frequency of each word raised to the power of 0.75, which is
    commonly used in algorithms like Word2Vec to moderate the influence of very frequent words.
    It then normalizes these adjusted frequencies to ensure they sum to 1, forming a probability
    distribution used for negative sampling.

    Parameters:
    - word_count_dict (dict): A dictionary where keys are words and values are the counts of those words.

    Returns:
    - dict: A dictionary where keys are words and values are the probabilities of selecting each word
            for negative sampling.

    Example:
    >>> word_counts = {'the': 5000, 'is': 1000, 'apple': 50}
    >>> get_negative_sampling_prob(word_counts)
    {'the': 0.298, 'is': 0.160, 'apple': 0.042}
    """

    ### YOUR CODE HERE
    count_power = {word: count ** 0.75 for word, count in word_count_dict.items()}
    Z = sum(count_power.values())
    negative_sampling_prob_dict = {
        word: value / Z  for word, value in count_power.items()
    }  # THIS IS A PLACEHOLDER!
    return negative_sampling_prob_dict

{'can': 0.005065333539070947,
 'i': 0.008562734270705188,
 'get': 0.002373264710127443,
 'back': 0.0004813961553108171,
 'with': 0.002598536055690065,
 'my': 0.0036108719977993925,
 'ex': 0.00017994572352561896,
 'even': 0.00042103868494002456,
 'though': 0.00015882076765360008,
 'she': 0.0005109214355176236,
 'is': 0.010406141117571129,
 'pregnant': 0.000173385765607508,
 'another': 0.0003144791215604004,
 'guys': 0.00022524681419348346,
 'baby': 0.0001444974335875018,
 'what': 0.011298113343445128,
 'are': 0.0064548883409054824,
 'some': 0.002481043468266601,
 'ways': 0.0005277504389414167,
 'to': 0.008504960393039253,
 'overcome': 0.00017700378020058712,
 'a': 0.009093128979994291,
 'fast': 0.00020699825450668646,
 'food': 0.00037218583219675484,
 'addiction': 7.130690048830135e-05,
 'who': 0.0017514496075980374,
 'were': 0.0006810837503990547,
 'the': 0.013109782896631204,
 'great': 0.0003218926371464949,
 'chinese': 0.00030208301666597185,
 'soldiers': 6.4540246358983e-05,
 'and':

Для удобства, преобразуем полученные словари в массивы (т.к. все слова все равно уже пронумерованы).

In [90]:
keep_prob_dict = subsample_frequent_words(word_count_dict)
assert keep_prob_dict.keys() == word_count_dict.keys()

In [91]:
negative_sampling_prob_dict = get_negative_sampling_prob(word_count_dict)
assert negative_sampling_prob_dict.keys() == negative_sampling_prob_dict.keys()
assert np.allclose(sum(negative_sampling_prob_dict.values()), 1)

In [92]:
keep_prob_array = np.array(
    [keep_prob_dict[index_to_word[idx]] for idx in range(len(word_to_index))]
)
negative_sampling_prob_array = np.array(
    [
        negative_sampling_prob_dict[index_to_word[idx]]
        for idx in range(len(word_to_index))
    ]
)

Если все прошло успешно, функция ниже поможет вам с генерацией подвыборок (батчей).

In [93]:
def generate_batch_with_neg_samples(
    context_pairs,
    batch_size,
    keep_prob_array,
    word_to_index,
    num_negatives,
    negative_sampling_prob_array,
):
    batch = []
    neg_samples = []

    while len(batch) < batch_size:
        center, context = random.choice(context_pairs)
        if random.random() < keep_prob_array[center]:
            batch.append((center, context))
            neg_sample = np.random.choice(
                range(len(negative_sampling_prob_array)),
                size=num_negatives,
                p=negative_sampling_prob_array,
            )
            neg_samples.append(neg_sample)
    batch = np.array(batch)
    neg_samples = np.vstack(neg_samples)
    return batch, neg_samples

In [139]:
batch_size = 4
num_negatives = 15
batch, neg_samples = generate_batch_with_neg_samples(
    context_pairs,
    batch_size,
    keep_prob_array,
    word_to_index,
    num_negatives,
    negative_sampling_prob_array,
)

Наконец, время реализовать модель. Обращаем ваше внимание, использование линейных слоев (`nn.Linear`) далеко не всегда оправданно!

Напомним, что в случае negative sampling решается задача максимизации следующего функционала:

$$
\mathcal{L} = \log \sigma({\mathbf{v}'_{w_O}}^\top \mathbf{v}_{w_I}) + \sum_{i=1}^{k} \mathbb{E}_{w_i \sim P_n(w)} \left[ \log \sigma({-\mathbf{v}'_{w_i}}^\top \mathbf{v}_{w_I}) \right],
$$

где:
- $\mathbf{v}_{w_I}$ – вектор центрального слова $w_I$,
- $\mathbf{v}'_{w_O}$ – вектор слова из контекста $w_O$,
- $k$ – число negative samplesЮ,
- $P_n(w)$ – распределение negative samples, заданное выше,
- $\sigma$ – сигмоида.

In [140]:
class SkipGramModelWithNegSampling(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.center_embeddings = nn.Embedding(vocab_size, embedding_dim) # YOUR CODE HERE
        self.context_embeddings = nn.Embedding(vocab_size, embedding_dim)  # YOUR CODE HERE

    def forward(self, center_words, pos_context_words, neg_context_words):
        # YOUR CODE HERE
        center_embeds = self.center_embeddings(center_words)
        pos_context_embeds = self.context_embeddings(pos_context_words)
        neg_context_embeds = self.context_embeddings(neg_context_words)

        pos_scores = torch.sum(center_embeds*pos_context_embeds, dim=-1)
        
        center_embeds_expanded = center_embeds.unsqueeze(1)

        neg_dot_products = torch.bmm(neg_context_embeds, center_embeds_expanded.transpose(1,2)).squeeze(2)
        neg_scores = neg_dot_products
      

        return pos_scores, neg_scores

In [141]:
device = torch.device("cpu")

In [142]:
vocab_size = len(word_to_index)
embedding_dim = 32
num_negatives = 30

model = SkipGramModelWithNegSampling(vocab_size, embedding_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.05)
lr_scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=150)
criterion = nn.BCEWithLogitsLoss()

In [143]:
params_counter = 0
for weights in model.parameters():
    params_counter += weights.shape.numel()
assert params_counter == len(word_to_index) * embedding_dim * 2

In [144]:
def train_skipgram_with_neg_sampling(
    model,
    context_pairs,
    keep_prob_array,
    word_to_index,
    batch_size,
    num_negatives,
    negative_sampling_prob_array,
    steps,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    device=device,
):
    pos_labels = torch.ones(batch_size).to(device)
    neg_labels = torch.zeros(batch_size, num_negatives).to(device)
    loss_history = []
    for step in tqdma(range(steps)):
        batch, neg_samples = generate_batch_with_neg_samples(
            context_pairs,
            batch_size,
            keep_prob_array,
            word_to_index,
            num_negatives,
            negative_sampling_prob_array,
        )
        center_words = torch.tensor([pair[0] for pair in batch], dtype=torch.long).to(
            device
        )
        pos_context_words = torch.tensor(
            [pair[1] for pair in batch], dtype=torch.long
        ).to(device)
        neg_context_words = torch.tensor(neg_samples, dtype=torch.long).to(device)

        optimizer.zero_grad()
        pos_scores, neg_scores = model(
            center_words, pos_context_words, neg_context_words
        )

        loss_pos = criterion(pos_scores, pos_labels)
        loss_neg = criterion(neg_scores, neg_labels)

        loss = loss_pos + loss_neg
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())
        lr_scheduler.step(loss_history[-1])

        if step % 100 == 0:
            print(
                f"Step {step}, Loss: {np.mean(loss_history[-100:])}, learning rate: {lr_scheduler._last_lr}"
            )

In [ ]:
steps = 750
batch_size = 2048
train_skipgram_with_neg_sampling(
    model,
    context_pairs,
    keep_prob_array,
    word_to_index,
    batch_size,
    num_negatives,
    negative_sampling_prob_array,
    steps,
)

  0%|          | 0/750 [00:00<?, ?it/s]

Step 0, Loss: 4.654812812805176, learning rate: [0.05]


Наконец, используйте полученную матрицу весов в качестве матрицы в векторными представлениями слов. Рекомендуем использовать для сдачи матрицу, которая отвечала за слова из контекста (т.е. декодера).

In [125]:
_model_parameters = model.parameters()
embedding_matrix_center = next(
    _model_parameters
).detach()  # Assuming that first matrix was for central word
embedding_matrix_context = next(
    _model_parameters
).detach()  # Assuming that second matrix was for context word

In [126]:
def get_word_vector(word, embedding_matrix, word_to_index=word_to_index):
    return embedding_matrix[word_to_index[word]]

Простые проверки:

In [127]:
similarity_1 = F.cosine_similarity(
    get_word_vector("iphone", embedding_matrix_context)[None, :],
    get_word_vector("apple", embedding_matrix_context)[None, :],
)
similarity_2 = F.cosine_similarity(
    get_word_vector("iphone", embedding_matrix_context)[None, :],
    get_word_vector("dell", embedding_matrix_context)[None, :],
)
assert similarity_1 > similarity_2

In [138]:
similarity_1 = F.cosine_similarity(
    get_word_vector("windows", embedding_matrix_context)[None, :],
    get_word_vector("laptop", embedding_matrix_context)[None, :],
)
similarity_2 = F.cosine_similarity(
    get_word_vector("windows", embedding_matrix_context)[None, :],
    get_word_vector("macbook", embedding_matrix_context)[None, :],
)
assert similarity_1 > similarity_2

AssertionError: 

Наконец, взглянем на ближайшие по косинусной мере слова. Функция реализована ниже.

In [129]:
def find_nearest(word, embedding_matrix, word_to_index=word_to_index, k=10):
    word_vector = get_word_vector(word, embedding_matrix)[None, :]
    dists = F.cosine_similarity(embedding_matrix, word_vector)
    index_sorted = torch.argsort(dists)
    top_k = index_sorted[-k:]
    return [(index_to_word[x], dists[x].item()) for x in top_k.numpy()]

In [130]:
find_nearest("python", embedding_matrix_context, k=10)

[('database', 0.770519495010376),
 ('ruby', 0.7728136777877808),
 ('mysql', 0.7733794450759888),
 ('tools', 0.7831018567085266),
 ('frontend', 0.7931641340255737),
 ('algorithm', 0.7969934940338135),
 ('php', 0.799119234085083),
 ('api', 0.8041577935218811),
 ('framework', 0.8589921593666077),
 ('python', 1.0000001192092896)]

Также вы можете визуально проверить, как представлены в латентном пространстве часто встречающиеся слова.

In [131]:
top_k = 5000
_top_words = sorted([x for x in word_count_dict.items()], key=lambda x: x[1])[
    -top_k - 100 : -100
]  # ignoring 100 most frequent words
top_words = [x[0] for x in _top_words]
del _top_words

In [132]:
word_embeddings = torch.cat(
    [embedding_matrix_context[word_to_index[x]][None, :] for x in top_words], dim=0
).numpy()

In [133]:
import bokeh.models as bm
import bokeh.plotting as pl
from bokeh.io import output_notebook

output_notebook()


def draw_vectors(
    x,
    y,
    radius=10,
    alpha=0.25,
    color="blue",
    width=600,
    height=400,
    show=True,
    **kwargs,
):
    """draws an interactive plot for data points with auxilirary info on hover"""
    if isinstance(color, str):
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({"x": x, "y": y, "color": color, **kwargs})

    fig = pl.figure(active_scroll="wheel_zoom", width=width, height=height)
    fig.scatter("x", "y", size=radius, color="color", alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show:
        pl.show(fig)
    return fig

Loading BokehJS ...

In [134]:
embedding = umap.UMAP(n_neighbors=5).fit_transform(word_embeddings)

In [135]:
draw_vectors(embedding[:, 0], embedding[:, 1], token=top_words)

figure(id='p1106', ...)

Для сдачи задания необходимо загрузить функции `subsample_frequent_words` и `get_negative_sampling_prob`, а также сгенерировать файл для посылки ниже и приложить в соответствующую задачу. Успехов!

In [136]:
# do not change the code in the block below
# __________start of block__________
import os
import json

assert os.path.exists(
    "words_subset.txt"
), "Please, download `words_subset.txt` and place it in the working directory"

with open("words_subset.txt") as iofile:
    selected_words = iofile.read().split("\n")


def get_matrix_for_selected_words(selected_words, embedding_matrix, word_to_index):
    word_vectors = []
    for word in selected_words:
        index = word_to_index.get(word, None)
        vector = [0.0] * embedding_matrix.shape[1]
        if index is not None:
            vector = embedding_matrix[index].numpy().tolist()
        word_vectors.append(vector)
    return word_vectors


word_vectors = get_matrix_for_selected_words(
    selected_words, embedding_matrix_context, word_to_index
)

with open("submission_dict.json", "w") as iofile:
    json.dump(word_vectors, iofile)
print("File saved to `submission_dict.json`")
# __________end of block__________

File saved to `submission_dict.json`
